# Test notebook
- https://huggingface.co/learn/agents-course/unit4/hands-on
- https://github.com/marketplace?type=models

In [1]:
pip install -r requirements.txt -qU

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
import inspect
import typing

from dotenv import load_dotenv
import pandas as pd
import gradio as gr


load_dotenv()

api_url = "https://agents-course-unit4-scoring.hf.space"
questions_url = f"{api_url}/questions"
random_question_url = f"{api_url}/random-question"
files_url = f"{api_url}/files"
submit_url = f"{api_url}/submit"

def get_question() -> typing.Tuple[str, str, bytes | None, str | None]:
    """
    Get a random question from the API.
    Returns
        - question text
        - task id
        - binary file (if any)
        - file name (if any)
    """

    question_data = requests.get(questions_url, timeout=15)
    question_data.raise_for_status()
    questions_json = question_data.json()[0]

    question = questions_json["question"]
    task_id = questions_json["task_id"]
    file_content = None

    file_name = questions_json.get("file_name", None)

    if file_name:
        file_url = f"{files_url}/{task_id}"
        response = requests.get(file_url, timeout=15)
        response.raise_for_status()
        # with open(os.path.join("tmp", file_name), "wb") as f:
        #     f.write(response.content)
        file_content = response.content

    return question, task_id, file_content, file_name

In [ ]:
question, task_id, file_content, file_name = get_question()

import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.github.ai/inference"
model_name = "microsoft/Phi-4"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        UserMessage(question),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print("Question:", question)
print("Response:", response.choices[0].message.content)

Question:  How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.
Response:  Between 2000 and 2009, Mercedes Sosa released eight studio albums. Here is a list of those albums:

1. **Mujeres Argentinas** (2000)
2. **Cantora** (2001)
3. **30 Años de Vida de Mercedes Sosa** (2004)
4. **Y cómo es él** (2005)
5. **Canta por los ausentes** (2006) – Although primarily distributed as a promotion album, it has significant content.
6. **Alta fidelidad** (2007)
7. **Rodolfo** (2008) – A collaboration with Joan Manuel Serrat.
8. **Cantora 2** (2009)

Please note that the released albums can vary in definition based on official releases versus compilations or promotional distributions, but these are the mainstream studio albums released within that time frame.
